# Atmospherically Corrected Time Series in Earth Engine

### Overview

This notebook creates an atmospherically corrected Sentinel 2 time series.

### Output

Mean average (cloud-free) pixel values.

#### Initialize

In [1]:
import ee
ee.Initialize()

In [2]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),'atmcorr'))
import interpolated_lookup_tables as iLUT
from ee_requests import request_cloudfree_averages
# import mission_specifics => solar_zenith_filter, band_names, mission_names
# import request_cloudfree_averages
# import atmcorr

#### Input variables

In [3]:
# region of interest
geom = ee.Geometry.Point(-157.815781, 21.292931).buffer(100)

# start and end of time series
startDate = ee.Date('2017-01-01')
stopDate  = ee.Date('2017-02-01')

#### Interpolated look-up tables

The emulator uses interpolated look-up tables to generate atmospheric correction coeffients.
They are downloaded automatically (if necessary).

In [4]:
iLUTs = iLUT.handler() 
iLUTs.iLUT_path = '/home/sam/github/6S_emulator/files/iLUTs/S2A_MSI/Continental/view_zenith_0'
iLUTs.get()  # must define one of following: imageCollectionID, satellite_mission, iLUT_path

#### Image Collection

In [5]:
ic = ee.ImageCollection('COPERNICUS/S2')\
  .filterBounds(geom)\
  .filterDate(startDate, stopDate)\
  .filter(ee.Filter.lt('MEAN_SOLAR_ZENITH_ANGLE',75))

#### Get cloud-free averages

In [6]:
# earth engine request
request = request_cloudfree_averages(ic, geom)

# get data locally (i.e. client side)
print('Getting info from Earth Engine.. ')
cloudfree_averages = request.getInfo()

Getting info from Earth Engine.. 


In [7]:
data = cloudfree_averages['features']
print(data[0]['properties'])

{'mean_averages': {'B1': 1427.0, 'B10': 9.0, 'B11': 1592.0, 'B12': 1230.0, 'B2': 1119.0, 'B3': 1109.0, 'B4': 1009.0, 'B5': 1078.0, 'B6': 1171.0, 'B7': 1301.0, 'B8': 1299.0, 'B8A': 1516.0, 'B9': 337.0, 'QA10': 0.0, 'QA20': 0.0, 'QA60': 0.0}}


#### Atmospheric Correction

In [8]:
# atmcorr_timeseries = atmcorr(cloudfree_averages, iLUTs)

#### Plot

In [9]:
# insert pretty plot here (before and after?)